In [ ]:
import numpy as np
import sys




# read from FASTA format file and convert it into string of sequence
def ReadFromFiles(file):
    f1 = open(file)
    s = f1.readlines()
    f1.close()
    s=s[1:]
    lines=""
    for line in s:
        line = line.strip()
        lines = lines+line

    return lines


# Print all the matrix even if its very big
np.set_printoptions(threshold=sys.maxsize)

# print every line of the matrix on one line
np.set_printoptions(linewidth=np.inf)

# make the output to scroll horizontally so we can have a nicely formatted score matrix using the above 2 numpy functions
from IPython.display import display, HTML
display(HTML("<style>pre { white-space: pre !important; }</style>"))

# Needleman-Wunsch Algorithm
class NeedlemanWunsch():
    def __init__(self,seq1,seq2,MatchScore,MismatchScore,GapScore):
        self.seq1 = seq1
        self.seq2 = seq2
        self.GapScore = GapScore
        self.MismatchScore = MismatchScore
        self.MatchScore = MatchScore


        diagonalScore = 0
        upScore = 0
        leftScore = 0

        # initialize the score Matrix and direction Matrix
        scoringMat = np.zeros((len(seq1)+1, len(seq2)+1),dtype=int)
        directionMat = np.zeros((len(seq1)+1, len(seq2)+1),dtype=object)



        #filling the first row
        for i in range(len(seq2)+1):
            if i+1<len(seq2)+1:

                scoringMat[0][i+1] = scoringMat[0][i] + GapScore

        #filling the first column
        for i in range(len(seq1)+1):
            if i+1<len(seq1)+1:

                scoringMat[i+1][0] = scoringMat[i][0] + GapScore
        #initialize of indices of the score matrix and direction matrix
        global index1
        global index2


        # filling the score matrix and the Direction matrix
        def scoreAndDirectMatrices():
            index1 = 1
            index2 = 1
            for i in seq1:
                for j in seq2:

                    if j == i:
                        diagonalScore = scoringMat[index1-1][index2-1] +MatchScore

                        upScore = scoringMat[index1-1][index2] + GapScore

                        leftScore = scoringMat[index1][index2-1] + GapScore

                        scoringMat[index1][index2] = max(diagonalScore,upScore,leftScore)
                                    

                        if scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2] = 'D, U, L'
                        elif scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] != leftScore:
                            directionMat[index1][index2] = 'D, U'
                        elif scoringMat[index1][index2] != diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2] = 'U, L'
                        elif scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] != upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2] = 'D, L'
                        elif scoringMat[index1][index2] == diagonalScore:
                            directionMat[index1][index2] = 'D'
                        elif scoringMat[index1][index2] == upScore:
                            directionMat[index1][index2] = 'U'
                        elif scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2] = 'L'



                    else:

                        diagonalScore = scoringMat[index1 - 1][index2 - 1] + MismatchScore
                        upScore = scoringMat[index1 - 1][index2] + GapScore
                        leftScore = scoringMat[index1][index2 - 1] + GapScore

                        scoringMat[index1][index2] = max(diagonalScore, upScore, leftScore)

                        if scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2]='D, U, L'
                        elif scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] != leftScore:
                            directionMat[index1][index2]='D, U'
                        elif scoringMat[index1][index2] != diagonalScore and scoringMat[index1][index2] == upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2]='U, L'
                        elif scoringMat[index1][index2] == diagonalScore and scoringMat[index1][index2] != upScore and scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2]='D, L'
                        elif scoringMat[index1][index2] == diagonalScore:
                            directionMat[index1][index2]='D'
                        elif scoringMat[index1][index2] == upScore:
                            directionMat[index1][index2]='U'
                        elif scoringMat[index1][index2] == leftScore:
                            directionMat[index1][index2]='L'

                    index2 += 1

                if index1 < len(seq1):
                    index1 += 1
                index2 = 1

        scoreAndDirectMatrices()

        # finding of indices that create the path in the matrix of score using the direction matrix

        indexOfOptAlign = []

        def indicesOfOptAlign():
            index1 = len(seq1)
            index2 = len(seq2)
            while(index1!=0 and index2!=0):
                indexOfOptAlign.append([index1,index2])

                leftScore = scoringMat[index1][index2-1]
                upScore = scoringMat[index1-1][index2]
                diagonalScore = scoringMat[index1-1][index2-1]
                if directionMat[index1][index2] == 'D, U, L':
                    m = max(leftScore,upScore,diagonalScore)
                    if m == leftScore:
                        index2 = index2-1
                    elif m == upScore:
                        index1 = index1 -1
                    elif m == diagonalScore:
                        index1 = index1 - 1
                        index2 = index2 -1
                elif directionMat[index1][index2] =='U, L':
                    m = max(leftScore,upScore)
                    if m == leftScore:
                        index2 = index2-1
                    elif m == upScore:
                        index1 = index1 -1

                elif directionMat[index1][index2] == 'D, U':
                    m = max(upScore,diagonalScore)

                    if m == upScore:
                        index1 = index1 -1
                    elif m == diagonalScore:
                        index1 = index1 - 1
                        index2 = index2 -1
                elif directionMat[index1][index2] == 'D, L':
                    m = max(leftScore,diagonalScore)
                    if m == leftScore:
                        index2 = index2-1

                    elif m == diagonalScore:
                        index1 = index1 - 1
                        index2 = index2 -1

                elif directionMat[index1][index2] == 'U':
                    index1 = index1 -1
                elif directionMat[index1][index2] =='L':
                    index2 = index2 -1
                elif directionMat[index1][index2] =='D':
                    index1 = index1 -1
                    index2 = index2 -1

        indicesOfOptAlign()
        # we reverse the indices of optimal alignment scores to have it from left to right
        indexOfOptAlign.reverse()


        # Finding the optimal alignment based on all what we did earlier
        def OptAlign():
            newSeq1 = seq1
            newSeq2 = seq2

            for i in range(len(indexOfOptAlign)):
                if i>0:
                    if indexOfOptAlign[i][1] == indexOfOptAlign[i-1][1]:
                        newSeq2 = newSeq2[:i] + '-' + newSeq2[i:]
                    if indexOfOptAlign[i][0] == indexOfOptAlign[i-1][0]:
                        newSeq1 = newSeq1[:i] + '-' + newSeq1[i:]

            return newSeq1 + "\n\n" + newSeq2



        # printing all the results


        print(scoringMat)
        output = open("output.txt" ,"w")
        output.write("Optimal Alignment: ")
        output.write(("\n"))
        output.write(OptAlign())
        output.close()
        print("\n")
        print("Score: " + str(scoringMat[len(seq1)][len(seq2)]) + "\n")
        print("Optimal Alignment: \n" + OptAlign())


# Homologous Gene comparisons
# NeedlemanWunsch(ReadFromFiles("Homologous1.fa"),ReadFromFiles("Homologous2.fa"),4,-2,0)
# NeedlemanWunsch(ReadFromFiles("Homologous1.fa"),ReadFromFiles("Homologous2.fa"),2,-1,-2)

# Human and Hamster Isuline Protein comparisons
# NeedlemanWunsch(ReadFromFiles("HamsterInsulinProtein.fa"),ReadFromFiles("HumanInsulinProtein.fa"),4,-2,0)
# NeedlemanWunsch(ReadFromFiles("HamsterInsulinProtein.fa"),ReadFromFiles("HumanInsulinProtein.fa"),2,-1,-2)




In [ ]:
NeedlemanWunsch(ReadFromFiles("Homologous1.fa"),ReadFromFiles("Homologous2.fa"),4,-2,0)

In [ ]:
NeedlemanWunsch(ReadFromFiles("Homologous1.fa"),ReadFromFiles("Homologous2.fa"),2,-1,-2)

In [ ]:
NeedlemanWunsch(ReadFromFiles("HamsterInsulinProtein.fa"),ReadFromFiles("HumanInsulinProtein.fa"),4,-2,0)

In [ ]:
NeedlemanWunsch(ReadFromFiles("HamsterInsulinProtein.fa"),ReadFromFiles("HumanInsulinProtein.fa"),2,-1,-2)